- Fazer o Web Scraping do site: https://books.toscrape.com/

- **Dados Necessários**:
    - Título
    - Preço
    - Nota
    - Disponibilidade
    - Categoria
    - Imagem

## Importando Bibliotecas Necessárias

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

## Requisitando a url

In [ ]:
url_books = 'https://books.toscrape.com/'
response = requests.get(url_books)

if response.status_code == 200:
    content = response.text
    print('Página obtida com sucesso.')
else:
    print(f'Erro ao acessar a página. Código de status: {response.status_code}')

Página obtida com sucesso.


## Pegando o conteúdo da página com o BeautifulSoup

In [ ]:
soup = BeautifulSoup(content, 'html.parser')

In [ ]:
# # Print da página inteira com o soup
# soup

## O que preciso de cada livro?


- **Todos os livros**: ol class="row" > find_all(li)

- **Título do livro**: h3 > a title
- **Preço**: div class="product_price" > p class="price_color"
- **Nota**: p class="star-rating X", sendo X a nota
- **Disponibilidade**: p class="instock availability" > i class="icon-ok"
- **Categoria**: Necessário seguir o link do livro até outra página, na outra página: ul class="breadcrumb" > li > a
- **Imagem**: div class="image_container" > a > img src="jpg"

### Definindo as queries necessárias para achar cada informação

In [ ]:
## Título
soup.find('article', class_='product_pod').find('h3').find('a')['title']

'A Light in the Attic'

In [ ]:
## Preço
# [2:] retira o símbolo de Libras
soup.find('div', class_='product_price').find('p', class_='price_color').text[2:]

'51.77'

In [ ]:
## Nota
soup.find('p', class_='star-rating')['class'][1]

'Three'

In [ ]:
## Disponibilidade
soup.find('p', class_='instock availability').find('i')['class'][0].split('-')[-1]

'ok'

In [ ]:
## Categoria passo 1
link_to_book = soup.find('article', class_='product_pod').find('h3').find('a')['href']
print(link_to_book)
full_link = requests.compat.urljoin(url_books, link_to_book)
full_link

catalogue/a-light-in-the-attic_1000/index.html


'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

In [ ]:
## Categoria passo 2
response_cat = requests.get(full_link)
content_cat = response_cat.text
soup_cat = BeautifulSoup(content_cat, 'html.parser')

In [ ]:
## Categoria passo 3
soup_cat.find('ul', class_='breadcrumb').find_all('li')[2].find('a').get_text()

'Poetry'

In [ ]:
## Imagem
img = soup.find('div', class_='image_container').find('a').find('img')['src']
img

'media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg'

In [ ]:
## Completando o link da imagem
requests.compat.urljoin(url_books, img)

'https://books.toscrape.com/media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg'

## Criando uma função para pegar a categoria e uma para pegar todos os outros dados

In [ ]:
def get_category(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    time.sleep(1)

    category = soup.find('ul', class_='breadcrumb').find_all('li')[2].find('a').get_text()

    return category

In [ ]:
def get_details(pages: int) -> list:
    """
    Pega os detalhes de cada livro dentro de cada página.

    Retorna uma lista de dicionários com os detalhes de cada livro.

    Args:
        pages (int): O número de páginas do site.

    Returns:
        list: Uma lista de dicionários com os detalhes de cada livro.
    """
    book_data = []

    for page in range(pages):
        url_raiz = 'https://books.toscrape.com/catalogue/'
        url = f'https://books.toscrape.com/catalogue/page-{page+1}.html'

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        books = soup.find('ol', class_='row').find_all('li')

        for book in books:
            title = book.find('article', class_='product_pod').find('h3').find('a')['title']
            price = book.find('div', class_='product_price').find('p', class_='price_color').text[2:]
            rating = book.find('p', class_='star-rating')['class'][1]
            availability = book.find('p', class_='instock availability').find('i')['class'][0].split('-')[-1]

            link_to_book = book.find('article', class_='product_pod').find('h3').find('a')['href']
            full_link = requests.compat.urljoin(url_raiz, link_to_book)

            category = get_category(full_link)
            img = book.find('div', class_='image_container').find('a').find('img')['src']
            image = requests.compat.urljoin(url_books, img)

            book_data.append({
                'Title': title,
                'Price': price,
                'Rating': rating,
                'Availability': availability,
                'Category': category,
                'Image': image
            })

        time.sleep(1)

    return book_data

## Achando o número de páginas

In [ ]:
pages = int(soup.find('li', class_='current').get_text(strip=True)[-2:])
pages

50

## Coletando os dados

In [ ]:
book_data = []

book_data = get_details(pages)

In [ ]:
df = pd.DataFrame(book_data)

In [ ]:
df.head()

,Title,Price,Rating,Availability,Category,Image
0,A Light in the Attic,51.77,Three,ok,Poetry,https://books.toscrape.com/media/cache/2c/da/2...
1,Tipping the Velvet,53.74,One,ok,Historical Fiction,https://books.toscrape.com/media/cache/26/0c/2...
2,Soumission,50.10,One,ok,Fiction,https://books.toscrape.com/media/cache/3e/ef/3...
3,Sharp Objects,47.82,Four,ok,Mystery,https://books.toscrape.com/media/cache/32/51/3...
4,Sapiens: A Brief History of Humankind,54.23,Five,ok,History,https://books.toscrape.com/media/cache/be/a5/b...


In [ ]:
df.shape

(1000, 6)

In [ ]:
df.to_csv('books.csv', index=False)